# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [29]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [30]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,san felipe,10.3399,-68.7425,21.84,100,13,0.56,VE,1724289910
1,1,acarau,-2.8856,-40.1200,25.40,84,1,8.33,BR,1724289843
2,2,belyy yar,53.6039,91.3903,15.94,94,0,1.00,RU,1724289910
3,3,nogliki,51.8333,143.1667,21.51,69,100,6.57,RU,1724289910
4,4,albany,42.6001,-73.9662,14.26,79,6,2.24,US,1724289910


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [31]:
# Configure the map
map_plot_2 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 600,
    size = "Humidity",
    scale = 0.8,
    color = "City",
    alpha = 0.5,
    hover_cols = ["City", "Country"]
)

# Display the map plot
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [48]:
# Narrow down cities that fit criteria and drop any results with null values
cities_criteria = city_data_df[(city_data_df['Humidity'] < 65) & 
                               (city_data_df['Wind Speed'] < 4.5) & 
                               (city_data_df['Cloudiness'] == 0) &
                               (city_data_df['Max Temp'] < 27)]

# Drop any rows with any null values
cities_criteria = cities_criteria.dropna()

# Display sample data
len(cities_criteria)


27

In [49]:
cities_criteria

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
65,65,hamilton,39.1834,-84.5333,19.67,60,0,2.06,US,1724289764
173,173,albertina,-22.2008,-46.6158,20.41,30,0,3.23,BR,1724289922
176,176,akcakale,36.7111,38.9475,26.71,64,0,3.46,TR,1724289856
178,178,letlhakeng,-24.0944,25.0298,12.75,28,0,1.23,BW,1724289922
220,220,sincik,38.0368,38.6149,22.73,25,0,1.78,TR,1724289925
254,254,qarah bagh,34.9402,61.7759,18.56,30,0,3.05,AF,1724289927
294,294,amol,36.4696,52.3507,23.04,57,0,0.48,IR,1724289863
301,301,tsiombe,-25.3000,45.4833,17.91,61,0,3.54,MG,1724289931
311,311,tshabong,-26.0500,22.4500,12.68,18,0,4.44,BW,1724289931
325,325,tejen,37.3834,60.5054,23.87,28,0,0.79,TM,1724289932


### Step 3: Create a new DataFrame called `hotel_df`.

In [50]:
# Use the Pandas copy function to create a DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = cities_criteria[['City', 'Country', 'Lng', 'Lat', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df.head()


,City,Country,Lng,Lat,Humidity,Hotel Name
65,hamilton,US,-84.5333,39.1834,60,
173,albertina,BR,-46.6158,-22.2008,30,
176,akcakale,TR,38.9475,36.7111,64,
178,letlhakeng,BW,25.0298,-24.0944,28,
220,sincik,TR,38.6149,38.0368,25,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [51]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "apiKey":geoapify_key 
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{row['Lng']},{row['Lat']},{radius}"
    params["bias"] = f"proximity:{row['Lng']},{row['Lat']}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

     # Add the category to the parameters
    params["categories"] = "accommodation.hotel"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
hamilton - nearest hotel: North Vista Manor
albertina - nearest hotel: Hotel Parque das Primaveras
akcakale - nearest hotel: No hotel found
letlhakeng - nearest hotel: No hotel found
sincik - nearest hotel: No hotel found
qarah bagh - nearest hotel: No hotel found
amol - nearest hotel: هتل شهر آمل
tsiombe - nearest hotel: No hotel found
tshabong - nearest hotel: No hotel found
tejen - nearest hotel: Tejen Myhmanhanasy
turuntayevo - nearest hotel: No hotel found
kone - nearest hotel: Pacifik Appartels
escanaba - nearest hotel: Magnuson Grand Pioneer Inn and Suites
lido di ostia - nearest hotel: Fly Decò Hotel
saint-hippolyte - nearest hotel: Ibis Perpignan nord
eregli - nearest hotel: Nupelda Caddesi
panjab - nearest hotel: Bazar Dan Siah Darah - بازار دان سیاه دره
kas - nearest hotel: KEKOVA OTEL
toliara - nearest hotel: Ambary
changji - nearest hotel: No hotel found
jincheng - nearest hotel: 德豪宾馆
kalaleh - nearest hotel: No hotel found
nova xavantina - nearest ho

,City,Country,Lng,Lat,Humidity,Hotel Name
65,hamilton,US,-84.5333,39.1834,60,North Vista Manor
173,albertina,BR,-46.6158,-22.2008,30,Hotel Parque das Primaveras
176,akcakale,TR,38.9475,36.7111,64,No hotel found
178,letlhakeng,BW,25.0298,-24.0944,28,No hotel found
220,sincik,TR,38.6149,38.0368,25,No hotel found
254,qarah bagh,AF,61.7759,34.9402,30,No hotel found
294,amol,IR,52.3507,36.4696,57,هتل شهر آمل
301,tsiombe,MG,45.4833,-25.3000,61,No hotel found
311,tshabong,BW,22.4500,-26.0500,18,No hotel found
325,tejen,TM,60.5054,37.3834,28,Tejen Myhmanhanasy


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [55]:
# remove records with no hotels found
hotel_df = hotel_df[hotel_df['Hotel Name']!="No hotel found"]
hotel_df

,City,Country,Lng,Lat,Humidity,Hotel Name
65,hamilton,US,-84.5333,39.1834,60,North Vista Manor
173,albertina,BR,-46.6158,-22.2008,30,Hotel Parque das Primaveras
294,amol,IR,52.3507,36.4696,57,هتل شهر آمل
325,tejen,TM,60.5054,37.3834,28,Tejen Myhmanhanasy
391,kone,NC,164.8658,-21.0595,41,Pacifik Appartels
399,escanaba,US,-87.0646,45.7453,59,Magnuson Grand Pioneer Inn and Suites
421,lido di ostia,IT,12.2765,41.7321,57,Fly Decò Hotel
443,saint-hippolyte,FR,2.9667,42.7833,60,Ibis Perpignan nord
447,eregli,TR,34.0467,37.5133,35,Nupelda Caddesi
456,panjab,AF,67.0233,34.3880,62,Bazar Dan Siah Darah - بازار دان سیاه دره


In [54]:
# Configure the map
map_plot_3 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 600,
    size = "Humidity",
    scale = 1.2,
    color = "City",
    alpha = 0.5,
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map plot
map_plot_3

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)